### Convolução usando Pytorch

Veremos em detalhe como realizar convolução usando Pytorch e como funciona a camada convolucional

In [15]:
import torch
from torch import nn
import torch.nn.functional as F

# Sinal
x = torch.tensor([5, 4, 8, 7, 9, 3, 6], dtype=torch.float32)
# Filtro
weight = torch.tensor([1, 2, 3], dtype=torch.float32)
# Tamanho do filtro
ks = len(weight)

# Redimensiona o sinal para o tamanho 1x1xlen(x). Ou seja, um batch contendo um único sinal, 
# e esse sinal possui um único canal
x = x.reshape(1,1,len(x))
# Redimensiona o filtro. O primeiro valor 1 possui um significado diferente do que
# no caso do sinal. Depois será explicado.
weight = weight.reshape(1,1,len(weight))
# Realiza a convolução
y = F.conv1d(x, weight)
y.shape

torch.Size([1, 1, 5])

O tamanho da entrada é 7 e da saída é 5. Isso porque o Pytorch realiza a convolução apenas nas posições que não necessitam de preenchimento de borda. Mas modificar o tamanho do resultado é indesejável. 

É muito comum realizarmos a convolução com padding para manter o tamanho do sinal:

In [16]:
# padding = ks//2 garante que a saída sempre terá o mesmo tamanho que a entrada
y = F.conv1d(x, weight, padding=ks//2)
print(y.shape)

torch.Size([1, 1, 7])


Dado o sinal [5, 4, 8, 7, 9, 3, 6] e filtro [1,2,3], nossa saída deve ser:

* y[0] = 1\*0 + 2\*5 + 3\*4 = 22
* y[1] = 1\*5 + 2\*4 + 3\*8 = 37
* ...
* y[6] = 1\*3 + 2\*6 + 3\*0 = 15

Note que a função realiza a correlação-cruzada, e não a convolução. Mas para redes neurais isso não importa.

In [17]:
print(y)

tensor([[[22., 37., 41., 49., 34., 33., 15.]]])


Em redes neurais a convolução possui o conceito de bias, que é simplesmente um valor constante que é adicionado ao resultado:

In [18]:
bias = torch.tensor([5.])
# Adiciona o valor 5 a cada elemento do resultado da convolução
y_bias = F.conv1d(x, weight, padding=ks//2, bias=bias)
print(y_bias)
print(torch.allclose(y+bias, y_bias))

tensor([[[27., 42., 46., 54., 39., 38., 20.]]])
True


### Camada de convolução

In [19]:
# A entrada terá 1 canal, queremos apenas 1 canal de saída. O tamanho do filtro é ks, o 
# padding é metade do tamanho do filtro e a camada não terá bias
conv = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=ks, padding=ks//2, bias=False)
y = conv(x)
print(y)

tensor([[[ 1.3683e+00,  2.2373e-01,  1.0971e+00, -2.2811e-04, -8.9165e-02,
          -1.3618e+00,  1.8204e-01]]], grad_fn=<ConvolutionBackward0>)


Uma camada de convolução consiste em um filtro possuindo valores aleatórios. Esse filtro possui o parâmetro requires_grad=True por padrão. Podemos alterar os valores do filtro se quisermos:

In [20]:
print(conv.weight.shape)
print(conv.weight)

with torch.no_grad():
    conv.weight[:] = weight

# Mesmo resultado que a convolução que fizemos antes:
conv(x)

torch.Size([1, 1, 3])
Parameter containing:
tensor([[[-0.2914,  0.1760,  0.1221]]], requires_grad=True)


tensor([[[22., 37., 41., 49., 34., 33., 15.]]], grad_fn=<ConvolutionBackward0>)

### Relação entre convolução e combinação linear

Uma convolução nada mais é do que uma combinação linear com menos parâmetros

In [22]:
# Matrix de convolução. Cada linha representa uma posição do kernel. Por exemplo,
# a linha 0 fará a operação 2*x[0]+3*x[1]+0*x[2]+...
matrix = torch.tensor([[2, 3, 0, 0, 0, 0, 0],
                       [1, 2, 3, 0, 0, 0, 0],
                       [0, 1, 2, 3, 0, 0, 0],
                       [0, 0, 1, 2, 3, 0, 0],
                       [0, 0, 0, 1, 2, 3, 0],
                       [0, 0, 0, 0, 1, 2, 3],
                       [0, 0, 0, 0, 0, 1, 2]], dtype=torch.float32)

# Aplica a matriz no sinal representado como um vetor coluna
(matrix@x.reshape(7,1)).T

tensor([[22., 37., 41., 49., 34., 33., 15.]])

Acima temos uma matrix 7x7 que recebe 7 atributos de entrada e gera 7 atributos de saída. Mas a combinação linear dos atributos de entrada sempre envolvem apenas 3 parâmetros.

### Convolução com mais de um canal

In [23]:
torch.random.manual_seed(42)
# Batch contendo um sinal de tamanho 7 e com 4 canais
x = torch.rand(size=(1,4,7))
# Camada que recebe sinal com 4 canais e gera um sinal com 5 canais.
conv = nn.Conv1d(in_channels=4, out_channels=5, kernel_size=ks, padding=ks//2, bias=False)

y = conv(x)
print('x\n',x)
# Saída possui tamanho 1x5x7
print('y\n',y)

x
 tensor([[[0.8823, 0.9150, 0.3829, 0.9593, 0.3904, 0.6009, 0.2566],
         [0.7936, 0.9408, 0.1332, 0.9346, 0.5936, 0.8694, 0.5677],
         [0.7411, 0.4294, 0.8854, 0.5739, 0.2666, 0.6274, 0.2696],
         [0.4414, 0.2969, 0.8317, 0.1053, 0.2695, 0.3588, 0.1994]]])
y
 tensor([[[ 0.3475, -0.0320,  0.2392,  0.2470,  0.0387,  0.1368, -0.0642],
         [-0.0022,  0.4083,  0.2041,  0.1272,  0.4129,  0.1850,  0.3437],
         [ 0.2184,  0.1856,  0.0187,  0.1791,  0.1456,  0.1202,  0.1170],
         [-0.1355, -0.1043, -0.0338, -0.1462, -0.0409, -0.2967, -0.1625],
         [-0.1107,  0.0499,  0.1445, -0.1307,  0.0096, -0.0671,  0.1102]]],
       grad_fn=<ConvolutionBackward0>)


* O **número de canais de saída** define o **número de fitlros** que serão utilizados na camada de convolução. No nosso caso, temos 5 filtros
* Cada filtro possui **tamanho espacial ks**
* Cada filtro possui **profundidade 4**, pois o sinal de entrada possui 4 canais.
* Portanto, temos 5 filtros de tamanho 4 x ks cada
* Portanto, o tamanho do tensor .weight da camada de convolução possui tamanho 5 x 4 x ks

In [24]:
conv.weight.shape

torch.Size([5, 4, 3])

In [10]:
# Filtro 1 da camada
filtro1 = conv.weight[0]
# Região do sinal que corresponde quando o filtro está na posição 1
regiao = x[0,:,0:3]
# Resultado da convolução para esse ponto específico
res = (filtro1*regiao).sum()
# comparação do resultado
print(torch.allclose(y[0,0,1], res))

True


### Outros parâmetros da convolução

#### Stride

In [25]:
x = torch.tensor([5, 4, 8, 7, 9, 3, 6], dtype=torch.float32)
weight = torch.tensor([1, 2, 3], dtype=torch.float32)
x = x.reshape(1,1,len(x))
weight = weight.reshape(1,1,len(weight))

# Faz o filtro deslocar duas posições ao invés de deslocar uma posição por vez
y = F.conv1d(x, weight, stride=2)
print(y)

tensor([[[37., 49., 33.]]])


In [26]:
# O resultado de stride=2 é equivalente a indexar a saída de stride=1 pulando 2 índices
yc = F.conv1d(x, weight, stride=1)
print(torch.allclose(y, yc[0,0,::2]))

True


#### Dilatação

In [27]:
y = F.conv1d(x, weight, dilation=2)
print(y)

tensor([[[48., 27., 44.]]])


In [29]:
# dilatação=2 é equivalente a inserir 0 entre os valores do filtro
wc = torch.tensor([1, 0, 2, 0, 3], dtype=torch.float32).reshape(1,1,-1)
yc = F.conv1d(x, wc, dilation=1)
print(torch.allclose(y, yc))

True
